## Goal
A/B tests play a huge role in website optimization. Analyzing A/B tests data is a very important
data scientist responsibility. Especially, data scientists have to make sure that results are
reliable, trustworthy, and conclusions can be drawn.
Furthermore, companies often run tens, if not hundreds, of A/B tests at the same time. Manually
analyzing all of them would require lot of time and people. Therefore, it is common practice to
look at the typical A/B test analysis steps and try to automate as much as possible. This frees
up time for the data scientists to work on more high level topics.
In this challenge, you will have to analyze results from an A/B test. Also, you will be asked to
design an algorithm to automate some steps.

## Challenge Description
Company XYZ is a worldwide e-commerce site with localized versions of the site.
A data scientist at XYZ noticed that Spain-based users have a much higher conversion rate than
any other Spanish-speaking country. She therefore went and talked to the international team in
charge of Spain And LatAm to see if they had any ideas about why that was happening.
Spain and LatAm country manager suggested that one reason could be translation. All Spanishspeaking
countries had the same translation of the site which was written by a Spaniard. They
agreed to try a test where each country would have its one translation written by a local. That is,
Argentinian users would see a translation written by an Argentinian, Mexican users by a Mexican
and so on. Obviously, nothing would change for users from Spain.
After they run the test however, they are really surprised cause the test is negative. I.e., it
appears that the non-localized translation was doing better!
You are asked to:
- Confirm that the test is actually negative. That is, it appears that the old version of the
- site with just one translation across Spain and LatAm performs better
- Explain why that might be happening. Are the localized translations really worse?
- If you identified what was wrong, design an algorithm that would return FALSE if the
same problem is happening in the future and TRUE if everything is good and the results
can be trusted.

In [1]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# load dataset
test_table = pd.read_csv("data/test_table.csv")
user_table = pd.read_csv("data/user_table.csv")
print(test_table.columns.values)
print(user_table.columns.values)

['user_id' 'date' 'source' 'device' 'browser_language' 'ads_channel'
 'browser' 'conversion' 'test']
['user_id' 'sex' 'age' 'country']


In [5]:
print(test_table.head())
print(user_table.head())

   user_id        date  source  device browser_language ads_channel  \
0   315281  2015-12-03  Direct     Web               ES         NaN   
1   497851  2015-12-04     Ads     Web               ES      Google   
2   848402  2015-12-04     Ads     Web               ES    Facebook   
3   290051  2015-12-03     Ads  Mobile            Other    Facebook   
4   548435  2015-11-30     Ads     Web               ES      Google   

       browser  conversion  test  
0           IE           1     0  
1           IE           0     1  
2       Chrome           0     0  
3  Android_App           0     1  
4      FireFox           0     1  
   user_id sex  age    country
0   765821   M   20     Mexico
1   343561   F   27  Nicaragua
2   118744   M   23   Colombia
3   987753   F   27  Venezuela
4   554597   F   20      Spain


In [27]:
data = pd.merge(test_table,user_table[['sex', 'age', 'country']],on=test_table.user_id, how = 'left')


KeyError: '[315281 497851 848402 ... 514870 785224 241662] not in index'

In [18]:
data.head()

,user_id,date,source,device,browser_language,ads_channel,browser,conversion,test,user_id,sex,age,country
user_id,,,,,,,,,,,,,
315281,742015.0,2015-12-01,Ads,Web,EN,Facebook,IE,0.0,1.0,510962.0,F,19.0,Mexico
497851,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
848402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
290051,338824.0,2015-12-04,Direct,Mobile,ES,NaN,Android_App,0.0,1.0,63860.0,F,19.0,Mexico
548435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
data.tail()

,user_id,date,source,device,browser_language,ads_channel,browser,conversion,test,user_id,sex,age,country
452862,247117,2015-12-03,SEO,Web,ES,NaN,Chrome,0,1,756215,F,27,Venezuela
452863,372871,2015-12-04,SEO,Web,Other,NaN,FireFox,0,0,36888,M,18,Argentina
452864,616082,2015-12-04,Ads,Web,ES,Google,Chrome,0,1,800559,M,28,Bolivia
452865,80466,2015-12-01,Ads,Mobile,ES,Google,Iphone_App,0,0,176584,M,19,Chile
452866,837203,2015-12-03,Ads,Mobile,ES,Facebook,Android_App,1,0,314649,M,24,Mexico


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452867 entries, 0 to 452866
Data columns (total 13 columns):
user_id             452867 non-null int64
date                452867 non-null object
source              452867 non-null object
device              452867 non-null object
browser_language    452867 non-null object
ads_channel         181676 non-null object
browser             452867 non-null object
conversion          452867 non-null int64
test                452867 non-null int64
user_id             452867 non-null int64
sex                 452867 non-null object
age                 452867 non-null int64
country             452867 non-null object
dtypes: int64(5), object(8)
memory usage: 44.9+ MB


In [16]:
data.describe()

,user_id,conversion,test,user_id,age
count,452867.000000,452867.000000,452867.000000,452867.000000,452867.000000
mean,499907.497283,0.049591,0.476445,499944.805166,27.130740
std,288666.992281,0.217098,0.499445,288676.264784,6.776678
min,1.000000,0.000000,0.000000,1.000000,18.000000
25%,249777.000000,0.000000,0.000000,249819.000000,22.000000
50%,499988.000000,0.000000,0.000000,500019.000000,26.000000
75%,749497.000000,0.000000,1.000000,749543.000000,31.000000
max,1000000.000000,1.000000,1.000000,1000000.000000,70.000000


In [28]:
 print (data[['test', 'conversion']].groupby('test', as_index=False).mean().sort_values(by='conversion', ascending=False))

   test  conversion
0   0.0     0.05537
1   1.0     0.04423


In [33]:
categories = ['source', 'device', 'browser_language', 'ads_channel',
 'browser']
for cate in categories:
    print (data[[cate, 'conversion']].groupby([cate], as_index=False).mean().sort_values(by='conversion', ascending=False))
    print ("_"*40)

   source  conversion
0     Ads    0.050173
2     SEO    0.050021
1  Direct    0.049944
________________________________________
   device  conversion
0  Mobile    0.050965
1     Web    0.049345
________________________________________
  browser_language  conversion
2            Other    0.053251
1               ES    0.050231
0               EN    0.048417
________________________________________
  ads_channel  conversion
1    Facebook    0.050963
4       Yahoo    0.050958
0        Bing    0.049636
2      Google    0.049544
3       Other    0.043840
________________________________________
       browser  conversion
5        Opera    0.056794
0  Android_App    0.051181
4   Iphone_App    0.050252
3           IE    0.049921
2      FireFox    0.049379
1       Chrome    0.049074
6       Safari    0.048006
________________________________________


In [39]:
temp_test = test_table[test_table.test==1]
temp_no_test = test_table[test_table.test==0]
categories = ['source', 'device', 'browser_language', 'ads_channel',
 'browser']
for cate in categories:
    print (temp_test[[cate, 'conversion']].groupby([cate], as_index=False).mean().sort_values(by=cate, ascending=True))
    print (temp_no_test[[cate, 'conversion']].groupby([cate], as_index=False).mean().sort_values(by=cate, ascending=True))
    print ("_"*40)

   source  conversion
0     Ads    0.043546
1  Direct    0.042489
2     SEO    0.043770
   source  conversion
0     Ads    0.055481
1  Direct    0.056221
2     SEO    0.054345
________________________________________
   device  conversion
0  Mobile    0.044003
1     Web    0.042959
   device  conversion
0  Mobile    0.055693
1     Web    0.054768
________________________________________
  browser_language  conversion
0               EN    0.042610
1               ES    0.043532
2            Other    0.044280
  browser_language  conversion
0               EN    0.053631
1               ES    0.055452
2            Other    0.054709
________________________________________
  ads_channel  conversion
0        Bing    0.042057
1    Facebook    0.044184
2      Google    0.043013
3       Other    0.037765
4       Yahoo    0.044876
  ads_channel  conversion
0        Bing    0.055060
1    Facebook    0.056777
2      Google    0.054737
3       Other    0.050113
4       Yahoo    0.055143
_________